## 1. dev data to e2e eval data  
qa form으로 만들기 위해서 답이 될 수 있는 output의 set을 list로 만들어야 한다.  

In [4]:
def combine_data(input_file_questions, input_file_answers, output_file, output_format='{input}\t{output_list}'):
    with open(input_file_questions, 'r') as f:
        questions = f.readlines()

    with open(input_file_answers, 'r') as f:
        answers = f.readlines()

    with open(output_file, 'w') as f:
        for q, a in zip(questions, answers):
            question = q.strip()
            answer_list = a.strip().split('\t')
            output_line = output_format.format(input=question, output_list=str(answer_list))
            f.write(output_line + '\n')

input_file_questions = "original/questions.txt"
input_file_answers = "original/answers.txt"
output_file = "gold_data.txt"
combine_data(input_file_questions, input_file_answers, output_file)


## 2. Split Data  
데이터를 train, validation, test set으로 나누어야 한다.  
이때, e2e와 retrieval evaluation을 편하게 하기 위해 문서를 많이 나누어야 한다.  

file1 = "original/biencoder-nq-dev.pages"
file2 = "original/biencoder-nq-dev.questions"
file3 = "gold_data.txt"

In [2]:
import random

def split_data(question_file, answer_file, gold_data_file, train_ratio=0.5, val_ratio=0.25, test_ratio=0.25):
    with open(question_file, 'r') as qf, open(answer_file, 'r') as af, open(gold_data_file, 'r') as gf:
        questions = qf.readlines()
        answers = af.readlines()
        gold_datas = gf.readlines()

    data = list(zip(questions, answers, gold_datas))
    random.shuffle(data)

    train_size = int(len(data) * train_ratio)
    val_size = int(len(data) * val_ratio)
    test_size = len(data) - train_size - val_size

    train_data = data[:train_size]
    val_data = data[train_size:train_size + val_size]
    test_data = data[train_size + val_size:]

    return train_data, val_data, test_data

def save_data(data, prefix):
    with open(f"{prefix}_question.txt", 'w') as qf, open(f"{prefix}_answer.txt", 'w') as af, open(f"{prefix}_retrieval.txt", 'w') as gf:
        for q, a, g in data:
            qf.write(q)
            af.write(a)
            gf.write(g)


question_file = "ext-source"  
answer_file = "ext-target"
gold_data_file = "ext-retrieval"

try:
    train_data, val_data, test_data = split_data(question_file, answer_file, gold_data_file)

    save_data(train_data, "train")
    save_data(val_data, "val")
    save_data(test_data, "test")

except FileNotFoundError:
    print(f"Error: File not found. Please check the file paths.")
except PermissionError:
    print(f"Error: Permission denied. Please check file permissions.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

save_data(train_data, "train")
save_data(val_data, "val")
save_data(test_data, "test")


## temp: list로 변환  


In [3]:
def combine_data(input_file_answers, output_file, output_format='{output_list}'):
    with open(input_file_answers, 'r') as f:
        answers = f.readlines()

    with open(output_file, 'w') as f:
        for a in answers:
            answer_list = a.strip().split('\t')
            output_line = output_format.format(output_list=str(answer_list))
            f.write(output_line + '\n')

input_file_answers = "./train_answer.txt"
output_file = "train.target"
combine_data(input_file_answers, output_file)

## 3. 데이터 무작위 추출  
데이터를 무작위로 추출하는 코드  

In [1]:
import random

def sample_data(files, num_samples=1000):
    """
    여러 파일에서 무작위로 샘플링된 데이터를 추출합니다.

    Args:
        files: 파일 경로 리스트
        num_samples: 추출할 샘플 수 (기본값: 200)

    Returns:
        list: 샘플링된 데이터 (각 요소는 파일별 데이터 리스트)
    """
    with open(files[0], "r", encoding="utf-8") as f0, \
         open(files[1], "r", encoding="utf-8") as f1, \
         open(files[2], "r", encoding="utf-8") as f2:

        lines = list(zip(f0, f1, f2))
        sampled_lines = random.sample(lines, num_samples)

        sampled_data = [
            [line[i].strip() for line in sampled_lines]
            for i in range(len(files))
        ]

    return sampled_data

def save_sampled_data(sampled_data, output_files):
    """
    샘플링된 데이터를 각각의 출력 파일에 저장합니다.

    Args:
        sampled_data: 샘플링된 데이터
        output_files: 출력 파일 경로 리스트
    """
    for i, data in enumerate(sampled_data):
        with open(output_files[i], "w", encoding="utf-8") as f:
            for line in data:
                f.write(f"{line}\n")

# 파일 경로 설정
input_files = ["source", "target", "retrieval"]
output_files = ["ext-source", "ext-target", "ext-retrieval"]

# 데이터 샘플링 및 저장
sampled_data = sample_data(input_files)
save_sampled_data(sampled_data, output_files)
